In [1]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector

import plotly.graph_objects as go

## Simulation

In this section, a simulation of violation of CHSH inequality is implemented.

In [2]:
simulator = AerSimulator()
shots = 1024

In [3]:
def gen_bell_Phi_p() -> QuantumCircuit:
    circuit = QuantumCircuit(2, 2)
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.cz(0, 1)
    return circuit

In [4]:
def gen_chsh_term(
    theta: float,
    phi: float,
) -> QuantumCircuit:
    circuit = gen_bell_Phi_p()
    circuit.rx(theta=-theta, qubit=0)
    circuit.rx(theta=-phi, qubit=1)

    return circuit

In [5]:
def observe_Z(
    circuit: QuantumCircuit,
    simulator: AerSimulator,
    shots: int,
) -> dict[str, int]:
    circuit.measure(0, 0)
    circuit.measure(1, 1)

    compiled_circuit = transpile(circuit, simulator)

    job = simulator.run(compiled_circuit, shots=shots)

    result = job.result()

    counts = result.get_counts(compiled_circuit)

    return counts

In [6]:
def calc_corr_term(counts: dict[str, int]) -> float:
    counts.setdefault('00', 0)
    counts.setdefault('01', 0)
    counts.setdefault('10', 0)
    counts.setdefault('11', 0)

    shots = sum(counts.values())

    p00 = counts['00'] / shots
    p01 = counts['01'] / shots
    p10 = counts['10'] / shots
    p11 = counts['11'] / shots

    return  p00 + p11 - p01 - p10

In [7]:
def calc_corr_through(
    theta: float,
    phi: float,
    simulator: AerSimulator = AerSimulator(),
    shots: int = 4000,
) -> float:
    circuit = gen_chsh_term(theta=theta, phi=phi)
    counts = observe_Z(circuit=circuit, simulator=simulator, shots=shots)
    corr = calc_corr_term(counts=counts)
    return corr

In [8]:
def calc_qCHSH_corr(
    t: float,
    t_: float,
    p: float,
    p_: float,
) -> float:
    li_directs_pairs = [(t, p), (t_, p), (t, p_), (t_, p_)]

    corrs = [calc_corr_through(*directs) for directs in li_directs_pairs]

    return sum(corrs) - 2 * corrs[2]

In [9]:
t = 0
t_ = - np.pi / 2.
p = - np.pi / 4.
p_ = - 3. * np.pi / 4.

In [10]:
num_sep = 8
ts = np.linspace(0, 2. * np.pi, num_sep + 1)
chshs_sim = []
for t in ts:
    chshs_sim.append(calc_qCHSH_corr(*(t, t_, p, p_)))

xs_sim = [i * np.pi / 4 for i in range(num_sep + 1)]
xs_text = [0 if i == 0 else fr'{i}π/4' for i in range(num_sep + 1)]

## Theoretical

In [11]:
xs_the = np.linspace(0, 2 * np.pi, 1000)
chshs_the = [
    np.cos(t - p) + np.cos(t_ - p) - np.cos(t - p_) + np.cos(t_ - p_)
    for t in xs_the
]

## Experimetal

In [12]:
def corr_add(
    a: float,
    b: float,
    c: float,
    d: float,
) -> float:
    return a + b - c + d

In [13]:
df_chshs = pd.read_csv('../out/data/chshs.csv', index_col=0)
corr_add(*df_chshs.iloc[0])

2.3440000000000003

In [14]:
chshs_exp = [corr_add(*df_chshs.loc[i]) for i in range(len(df_chshs))]

In [17]:
fig = go.Figure()

fig.add_hline(
    y = 2 * np.sqrt(2),
    line=dict(
        dash='dot',
        color='#AB63FA',
    )
)
fig.add_hline(
    y = 2,
    line=dict(
        dash='dot',
        color='#AB63FA',
    )
)
fig.add_hrect(
    y0=2., y1=2 * np.sqrt(2), line_width=0,
    fillcolor='#AB63FA',
    opacity=.2
)
fig.update_layout(
    yaxis={
        "tickvals": [0, 1, 2, 2*np.sqrt(2),3],
        "ticktext": [0, 1, 2, '2√2', 3]
    }
)
fig.add_trace(
    go.Scatter(
        x=xs_the, y=chshs_the, mode='lines',
        line=dict(
            dash='dot',
            width=1,
            color='black',
        ),
        name='theoretical'
    )
)
fig.add_trace(
    go.Scatter(
        x=xs_sim, y=chshs_sim, mode='lines+markers',
        line=dict(
            dash='dash',
            width=1.,
            color='#636EFA'
        ),
        marker=dict(
            size=7,
            color='#636EFA',
        ),
        name='simulation',
    )
)
fig.add_trace(
    go.Scatter(
        x=xs_sim, y=chshs_exp[:9],
        line=dict(
            dash='dash',
            width=1.,
            color='#EF553B',
        ),
        marker=dict(
            symbol='star',
            size=13,
            color='#EF553B',
        ),
        name='experiment',
    )
)
fig_size = 50
fig.update_layout(
    font_size=18,
    margin=dict(
        t=10,
        b=20,
        l=30,
        r=40,
    ),
    width=16 * fig_size, height=7 * fig_size,
    xaxis=dict(
        title='θ',
        tickmode='array',
        tickvals=xs_sim,
        ticktext=xs_text,
    ),
    yaxis=dict(
        title='<S>',
    )
)

In [18]:
fig.write_html("./../out/fig/CHSH.html")
fig.write_image("./../out/fig/CHSH.png", scale=2)